# Bittrex API Function Library

## Connect to Bittrex API

First we need to import the necessary libraries 

In [113]:
import requests       # for making http requests to bittrex 
import json           # for parsing what bittrex sends back to us 
import pandas as pd   # for storing and manipulating the data we get back 
import numpy as np    # for managing and transforming data 
import time           # for nonces and time management
import hmac           # for hashing api message as signatures
import hashlib        # library of hashing options 
import matplotlib.pyplot as plt              # for displaying data
import datetime as dt                        # for handling time 
from urllib.parse import urljoin, urlencode  # for structuring and signing api calls 


### Define the API base url 

In [40]:
# Bittrex API url 
BASE_URL = 'https://api.bittrex.com'

### Define BittrexException if call fails 

In [41]:
class BittrexException(Exception): 
    def __init__(self, status_code, data): 
        self.status_code = status_code
        if data:                            # if error has data, populate to display 
            self.code = data['code']
            self.msg = data['msg']
        else: 
            self.code = None
            self.msg = None
        message = f"{status_code} [{self.code}] {self.msg}"

### Retrieve open and available trading markets at Bittrex


In [51]:
# GET /public/getmrkets
def get_markets():
    PATH = '/api/v1.1/public/getmarkets'     # target of API Call 
    params = None

    url = urljoin(BASE_URL, PATH)            # creates path for api call
    r = requests.get(url, params = params)   # makes api call 
    if r.status_code == 200:                 # if successful, defines data
        data = r.json()
        df = pd.DataFrame(data['result'])
        print(df)
    else:                                    # else, raises exception
        raise BittrexException(status_code=r.status_code, data=r.json())
                               
# run function to see output  
get_markets()

    MarketCurrency BaseCurrency MarketCurrencyLong BaseCurrencyLong  \
0              LTC          BTC           Litecoin          Bitcoin   
1             DOGE          BTC           Dogecoin          Bitcoin   
2              VTC          BTC           Vertcoin          Bitcoin   
3              PPC          BTC           Peercoin          Bitcoin   
4              FTC          BTC        Feathercoin          Bitcoin   
..             ...          ...                ...              ...   
332           HBAR         USDT   Hedera Hashgraph           Tether   
333            PLG          BTC        Pledge Coin          Bitcoin   
334            VET          BTC            VeChain          Bitcoin   
335            VET         USDT            VeChain           Tether   
336            SIX          BTC         SixNetwork          Bitcoin   

     MinTradeSize MarketName  IsActive  IsRestricted                  Created  \
0        0.032972    BTC-LTC      True         False      2014-02-

### Retreive supported currencies at Bittrex

In [52]:
# GET /public/getcurrencies
def get_currencies(): 
    PATH = '/api/v1.1/public/getcurrencies'           
    params = None
    url = urljoin(BASE_URL, PATH)                   # define total path
    r = requests.get(url, params = params)          # add parameters and request
    if r.status_code == 200: 
        data = r.json()                             # parse data to json format
        df = pd.DataFrame(data['result'])           # convert json to data frame (only result)
        print(df) 
    else: 
        raise BittrexException(status_code= r.status_code(), data = r.json())
        
# run function to see output 
get_currencies() 

    Currency      CurrencyLong  MinConfirmation   TxFee  IsActive  \
0        BTC           Bitcoin                2  0.0005      True   
1        LTC          Litecoin                6  0.0100      True   
2       DOGE          Dogecoin                6  2.0000      True   
3        VTC          Vertcoin              600  0.0200      True   
4        PPC          Peercoin               12  0.0200      True   
..       ...               ...              ...     ...       ...   
272     MRPH  Morpheus.Network               36  1.0000      True   
273     HBAR  Hedera Hashgraph                1  0.1000      True   
274      VET           VeChain               20  0.0100      True   
275      PLG       Pledge Coin               36  1.0000      True   
276      SIX       Six Network                2  0.1000      True   

     IsRestricted      CoinType  \
0           False       BITCOIN   
1           False     BITCOIN16   
2           False       BITCOIN   
3           False     BITCOIN16

### Retrieve tick value for a market
Has parameters: 
* market: string (required)


In [96]:
# GET /public/getticker
def get_ticker(market):
    PATH = '/api/v1.1/public/getticker'
    params = {                                  # define parameters as dictionary 
        'market': market
    }
    url = urljoin(BASE_URL, PATH)
    r = requests.get(url, params = params)      # add parameters and request
    if r.status_code == 200: 
        data = r.json()
        df = pd.DataFrame(data)
        print(df)
    else: 
        raise BittrexException(status_code = r.status_code(), data = r.json())
        
# run function to see output
get_ticker('BTC-LTC')
get_ticker('BTC-ETH')


      success message    result
Ask      True          0.006273
Bid      True          0.006266
Last     True          0.006262
      success message    result
Ask      True          0.019707
Bid      True          0.019698
Last     True          0.019710


### Retrieve 24 hr summary of all markets

In [97]:
# GET /public/getmarketsummaries
def get_marketsummaries():
    PATH = '/api/v1.1/public/getmarketsummaries'        # code structure similar to above, just change path 
    params = None
    
    url = urljoin(BASE_URL, PATH)
    r = requests.get(url, params = params)
    if r.status_code == 200: 
        data = r.json()
        df = pd.DataFrame(data['result'])
        print(df)
    else: 
        raise BittrexException(status_code = r.status_code(), data = r.json())
        
# run function to see output
get_marketsummaries()


    MarketName          High           Low        Volume          Last  \
0      BTC-XMY  1.100000e-07  9.000000e-08  1.167461e+06  1.000000e-07   
1      ETH-XLM  3.879500e-04  3.764400e-04  6.667568e+05  3.879500e-04   
2     USDT-XRP  2.973405e-01  2.913741e-01  2.456340e+05  2.957350e-01   
3      BTC-XST  1.019000e-05  9.080000e-06  1.009999e+05  9.140000e-06   
4      BTC-LBC  8.800000e-07  8.400000e-07  3.805085e+05  8.400000e-07   
..         ...           ...           ...           ...           ...   
332    BTC-GNO  1.710000e-03  1.700000e-03  3.677000e+00  1.710000e-03   
333    ETH-REP  5.600973e-02  4.992908e-02  5.212164e+02  5.408902e-02   
334    ETH-XRP  1.622560e-03  1.595910e-03  4.422417e+04  1.616490e-03   
335  BTC-BLOCK  1.250000e-04  1.031500e-04  5.032494e+03  1.250000e-04   
336    BTC-EDG  1.250000e-06  1.230000e-06  5.967923e+03  1.230000e-06   

       BaseVolume                TimeStamp           Bid           Ask  \
0        0.120903  2019-11-03T00:56:0

### Retrieve market summary of specific market 
Has parameters: 
* market: string (required)

In [94]:
# GET /public/getmarketsummary
def get_marketsummaries(market):
    PATH = '/api/v1.1/public/getmarketsummary'
    params = {
        'market': market
    }
    url = urljoin(BASE_URL, PATH)
    r = requests.get(url, params = params)
    if r.status_code == 200: 
        data = r.json()
        df = pd.DataFrame(data['result'])
        print(df)
    else: 
        raise BittrexException(status_code = r.status_code(), data = r.json())
        
# run function to see output
get_marketsummaries('BTC-LTC')

  MarketName      High       Low       Volume      Last  BaseVolume  \
0    BTC-LTC  0.006335  0.006238  1706.243198  0.006262   10.716832   

                TimeStamp       Bid       Ask  OpenBuyOrders  OpenSellOrders  \
0  2019-11-03T00:54:57.69  0.006266  0.006273            512            3789   

    PrevDay              Created  
0  0.006304  2014-02-13T00:00:00  


### Retrieve order book for a given market 
Has Parameters: 
* market: string: buy || sell || both (required) 
* type: string (required) 

In [101]:
# GET /public/getorderbook 
def get_orderbook(market, book): 
    PATH = '/api/v1.1/public/getorderbook'
    params = {                                         # multiple params, add to dictionary
        'market': market, 
        'type': book
    }
    url = urljoin(BASE_URL, PATH)
    r = requests.get(url, params = params)
    if r.status_code == 200: 
        data = r.json()
        df = pd.DataFrame(data['result'])
        print(df)
    else: 
        raise BittrexException(status_code = r.status_code(), data = r.json())
        
# run function to see output
get_orderbook('BTC-LTC', 'both')

    

                                              buy  \
0     {'Quantity': 2.74346931, 'Rate': 0.0062655}   
1         {'Quantity': 10.3509, 'Rate': 0.006255}   
2   {'Quantity': 16.07992029, 'Rate': 0.00625354}   
3          {'Quantity': 0.56, 'Rate': 0.00625248}   
4    {'Quantity': 11.94252736, 'Rate': 0.0062519}   
..                                            ...   
95    {'Quantity': 1.20607571, 'Rate': 0.0060005}   
96       {'Quantity': 17.92341505, 'Rate': 0.006}   
97   {'Quantity': 6.65740081, 'Rate': 0.00599333}   
98           {'Quantity': 0.084, 'Rate': 0.00599}   
99      {'Quantity': 0.47128, 'Rate': 0.00598839}   

                                             sell  
0   {'Quantity': 72.06035236, 'Rate': 0.00626686}  
1   {'Quantity': 31.94751231, 'Rate': 0.00626687}  
2   {'Quantity': 47.92066026, 'Rate': 0.00626688}  
3    {'Quantity': 47.9207643, 'Rate': 0.00626689}  
4    {'Quantity': 10.02992029, 'Rate': 0.0062669}  
..                                            ...  

### Retrieve Market History 
Has parameters: 
* market: string (required) 

In [112]:
# GET /public/getmarkethistory
def get_orderbook(market): 
    PATH = '/api/v1.1/public/getmarkethistory'
    params = {
        'market': market 
    }
    url = urljoin(BASE_URL, PATH)
    r = requests.get(url, params = params)
    if r.status_code == 200: 
        data = r.json()
        df = pd.DataFrame(data['result'])
        print(df)
    else: 
        raise BittrexException(status_code = r.status_code(), data = r.json())
        
# run function to see output
get_orderbook('BTC-ETH')

    

          Id               TimeStamp  Quantity     Price     Total FillType  \
0   44593466  2019-11-03T02:33:25.99  0.586576  0.019746  0.011583     FILL   
1   44593461   2019-11-03T02:33:07.4  0.463017  0.019746  0.009143     FILL   
2   44593292  2019-11-03T02:28:12.68  5.267000  0.019751  0.104028     FILL   
3   44593226  2019-11-03T02:26:21.26  5.232100  0.019746  0.103313     FILL   
4   44593088  2019-11-03T02:23:02.52  0.080963  0.019740  0.001598     FILL   
..       ...                     ...       ...       ...       ...      ...   
95  44585276   2019-11-02T21:52:28.3  0.030440  0.019679  0.000599     FILL   
96  44585271  2019-11-02T21:52:04.19  0.169560  0.019679  0.003337     FILL   
97  44585099  2019-11-02T21:44:03.94  0.051934  0.019672  0.001022     FILL   
98  44585077  2019-11-02T21:42:47.67  0.125135  0.019663  0.002460     FILL   
99  44584980  2019-11-02T21:38:43.16  0.283163  0.019659  0.005567     FILL   

   OrderType                                  Uuid 

### Test API Signature Process

In [175]:
# Testing the API Signature Setup with path, api, and secret from Bittrex docs

PATH = '/api/v1.1/path/call'
params = {
    'param': 'value',
    'apikey': '0123456789'
}

secret = 'ABCDEFGHIJ'
message = urljoin(BASE_URL, PATH)
message = message + '?' + urlencode(params) + '&nonce=1542020339856'
signature = hmac.new(secret.encode('ascii'), message.encode('ascii'), hashlib.sha512).hexdigest()
expectedSig = "FC70DD6467C916FBAA14EB89942644F9CF3F4CBF179923EE677413DC2E1CF2D8C5AA60C85A0A0BB3174960B8AA4FCFB0BC20CDE19BF21D567C671811C52CE631"

if signature.lower() == expectedSig.lower():     # case insensitive comparison
    print('success')
else: 
    print('failed')
    

success


### API Signature function 
Module used to create signature

In [179]:
# params: url, params, secret
def signAPI(url, params, secret):
    message = url + '?' + urlencode(params)      # adds parameters per message sig requirements
    signature = hmac.new(secret.encode('ascii'), message.encode('ascii'), hashlib.sha512).hexdigest()
    return(signature)


### Place a buy order in a specific market 
Requires an API Key which has enabled trading 

Has parameters
* market: string (requred)
* quantity: decimal (required) 
* rate: decimal (requried) 
* timeInForce: string (not required) 


In [172]:
# GET /market/buylimit 

# market (formated X-Y), quantity (in terms of Y), rate (in terms of X)
def place_buylimit(market, quantity, rate, timeInForce=''): 
    PATH = '/api/v1.1/market/buylimit'
    
    # apikey and nonce required for signed transactions, add to dictionary 
    params = {                                             
        'apikey':'INPUT_APIKEY',
        'market': market, 
        'quantity': quantity, 
        'rate': rate,
        'nonce': str(time.time())
    }
    
    secret = 'INPUT_SECRET'                         # secret to be included in signature
    url = urljoin(BASE_URL, PATH)                   # url without parameters (needed for api request)
    signature = signAPI(url, params, secret)        # returns the signature for api call 
    
    # request includes url, parameters as seen before, includes api signature in the request header
    r = requests.get(url, params = params, headers={'apisign':signature})
    if r.status_code == 200: 
        data = r.json()
        print(data)
    else: 
        raise BittrexException(status_code = r.status_code(), data = r.json())
        
# run function to see output
place_buylimit('BTC-ETH', 1, 0.019)
    

https://api.bittrex.com/api/v1.1/market/buylimit?apikey=a495672ff7b7460196300acff578548f&market=BTC-ETH&quantity=1&rate=0.019&nonce=1572752802.420407
{'success': True, 'message': '', 'result': {'uuid': '239e3bd6-f8e0-4943-8f57-348406a936f8'}}


### Place a sell order in a specific market 
Requires an API Key which has enabled trading 

Has parameters
* market: string (requred)
* quantity: decimal (required) 
* rate: decimal (requried) 
* timeInForce: string (not required) 


In [ ]:
# GET /market/selllimit

# market (formated 'X-Y'), quantity (in terms of Y), rate (in terms of X)
def place_selllimit(market, quantity, rate, timeInForce=''): 
    PATH = '/api/v1.1/market/selllimit'
    
    # apikey and nonce required for signed transactions, add to dictionary 
    params = {                                             
        'apikey':'INPUT_APIKEY',
        'market': market, 
        'quantity': quantity, 
        'rate': rate,
        'nonce': str(time.time())                    # use time in current epoch as nonce
    }
    
    secret = 'INPUT_SECRET'                         # secret to be included in signature
    url = urljoin(BASE_URL, PATH)                   # url without parameters (needed for api request)
    signature = signAPI(url, params, secret)        # returns the signature for api call 
    
    # request includes url, parameters as seen before, includes api signature in the request header
    r = requests.get(url, params = params, headers={'apisign':signature})
    if r.status_code == 200: 
        data = r.json()
        print(data)
    else: 
        raise BittrexException(status_code = r.status_code(), data = r.json())
        
# run function to see output
place_selllimit('BTC-ETH', 1, 0.019)


### Place cancel of a buy or sell order
Requires an key with transaction approval and signature using that key 

Has parameters:
   * uuid: string (required): uuid of buy or sell order

In [ ]:
# GET /market/cancel

# uuid (string) of the order placed 
def place_cancel(uuid): 
    PATH = '/api/v1.1/market/cancel'
    
    # apikey and nonce required for signed transactions, add to dictionary 
    params = {                                             
        'apikey':'INPUT_APIKEY',
        'uuid': uuid,
        'nonce': str(time.time())                   # use time in current epoch as nonce
    }
    
    secret = 'INPUT_SECRET'                         # secret to be included in signature
    url = urljoin(BASE_URL, PATH)                   # url without parameters (needed for api request)
    signature = signAPI(url, params, secret)        # returns the signature for api call 
    
    # request includes url, parameters as seen before, includes api signature in the request header
    r = requests.get(url, params = params, headers={'apisign':signature})
    if r.status_code == 200: 
        data = r.json()
        print(data)
    else: 
        raise BittrexException(status_code = r.status_code(), data = r.json())
        
# run function to see output
place_cancel('uuid here')

### Retreive your Open Orders (A specific market can be requested)
Requires API key with market access and associated API key signature

Has parameters: 
 * market: string (not required)

In [184]:
# GET /market/getopenorders

def get_openorders(market = None): 
    PATH = '/api/v1.1/market/getopenorders'
    
    # need to check if a specifc market is input
    if market == None:
        params = {
            'apikey': 'eb90f7ce6dad485c8a852fb957f48ff3',
            'nonce' : str(time.time())
        }
    else:
        params = {                                             
        'apikey':'eb90f7ce6dad485c8a852fb957f48ff3',
        'market': market,
        'nonce': str(time.time())                   # use time in current epoch as nonce
    } 
    
    secret = '1b297b979337407bba31b00989dbdc19'     # secret to be included in signature
    url = urljoin(BASE_URL, PATH)                   # url without parameters (needed for api request)
    signature = signAPI(url, params, secret)        # returns the signature for api call 
    
    # request includes url, parameters as seen before, includes api signature in the request header
    r = requests.get(url, params = params, headers={'apisign':signature})
    if r.status_code == 200: 
        data = r.json()
        print(data)
    else: 
        raise BittrexException(status_code = r.status_code(), data = r.json())
        
# run function to see output
get_openorders()

{'success': True, 'message': '', 'result': []}


### Retrieve all balances from your account 
Requires API Key read access 

In [187]:
# GET /account/getbalances

def get_openorders(): 
    PATH = '/api/v1.1/account/getbalances'

    params = {
        'apikey': 'eb90f7ce6dad485c8a852fb957f48ff3',
        'nonce' : str(time.time())
    }

    
    secret = '1b297b979337407bba31b00989dbdc19'     # secret to be included in signature
    url = urljoin(BASE_URL, PATH)                   # url without parameters (needed for api request)
    signature = signAPI(url, params, secret)        # returns the signature for api call 
    
    # request includes url, parameters as seen before, includes api signature in the request header
    r = requests.get(url, params = params, headers={'apisign':signature})
    if r.status_code == 200: 
        data = r.json()
        df = pd.DataFrame(data['result'])
        print(df)
    else: 
        raise BittrexException(status_code = r.status_code(), data = r.json())
        
# run function to see output
get_openorders()

  Currency   Balance  Available  Pending                       CryptoAddress
0      BTC  0.187167   0.187167      0.0  1DmzDwfZX2G2yZ7Y9P13xc6JcHSE9zZ2w8
1      BTS  0.003425   0.003425      0.0                                None


### Retrieve balance of specific currency
Requires an API Key with read access

Has Parameters: 
* currency: string (required)

In [192]:
# GET /account/getbalance

def get_balance(currency): 
    PATH = '/api/v1.1/account/getbalance'

    params = {
        'apikey': 'eb90f7ce6dad485c8a852fb957f48ff3',
        'currency': currency,
        'nonce' : str(time.time())
    }

    
    secret = '1b297b979337407bba31b00989dbdc19'     # secret to be included in signature
    url = urljoin(BASE_URL, PATH)                   # url without parameters (needed for api request)
    signature = signAPI(url, params, secret)        # returns the signature for api call 
    
    # request includes url, parameters as seen before, includes api signature in the request header
    r = requests.get(url, params = params, headers={'apisign':signature})
    if r.status_code == 200: 
        data = r.json()
        print(data['result'])
    else: 
        raise BittrexException(status_code = r.status_code(), data = r.json())
        
# run function to see output
get_balance("BTC")

{'Currency': 'BTC', 'Balance': 0.18716665, 'Available': 0.18716665, 'Pending': 0.0, 'CryptoAddress': '1DmzDwfZX2G2yZ7Y9P13xc6JcHSE9zZ2w8'}


### Retrieve or Generate an address for specific currency
If an address doesn't exist, the call will fail and return ADDRESS_GENERATING until one is available 

Requires and APIKey with Read access 

Has parameters: 
* currency: string (required) 


In [195]:
# GET /account/getdepositaddress

def get_depositaddress(currency): 
    PATH = '/api/v1.1/account/getdepositaddress'

    params = {
        'apikey': 'eb90f7ce6dad485c8a852fb957f48ff3',
        'currency': currency,
        'nonce' : str(time.time())
    }

    
    secret = '1b297b979337407bba31b00989dbdc19'     # secret to be included in signature
    url = urljoin(BASE_URL, PATH)                   # url without parameters (needed for api request)
    signature = signAPI(url, params, secret)        # returns the signature for api call 
    
    # request includes url, parameters as seen before, includes api signature in the request header
    r = requests.get(url, params = params, headers={'apisign':signature})
    if r.status_code == 200: 
        data = r.json()
        print(data['result'])
    else: 
        raise BittrexException(status_code = r.status_code(), data = r.json())
        
# run function to see output
get_depositaddress("BTC")

{'Currency': 'LTC', 'Address': 'LSBHPMqsJPV87VgqaAd2XS7D8RQwaDcu6u'}


### There's a withdraw api call.... not gonna mess with that for now


### Retrieve a single order using uuid
Requires an API Key with Read access

Has parameters:
   * uuid: string (required)

In [ ]:
# GET /account/getorder

def get_order(uuid): 
    PATH = '/api/v1.1/account/getorder'

    params = {
        'apikey': 'eb90f7ce6dad485c8a852fb957f48ff3',
        'uuid': uuid,
        'nonce' : str(time.time())
    }

    
    secret = '1b297b979337407bba31b00989dbdc19'     # secret to be included in signature
    url = urljoin(BASE_URL, PATH)                   # url without parameters (needed for api request)
    signature = signAPI(url, params, secret)        # returns the signature for api call 
    
    # request includes url, parameters as seen before, includes api signature in the request header
    r = requests.get(url, params = params, headers={'apisign':signature})
    if r.status_code == 200: 
        data = r.json()
        print(data['result'])
    else: 
        raise BittrexException(status_code = r.status_code(), data = r.json())
        
# run function to see output
get_order("uuid here")

### Retrieve order history 
Requires an API Key with Read access

Has Parameters: 
* market: string (not required)

In [203]:
# GET /account/getorderhistory

def get_orderhistory(market = None): 
    PATH = '/api/v1.1/account/getorderhistory'
    
    # need to check if a specifc market is input
    if market == None:
        params = {
            'apikey': 'eb90f7ce6dad485c8a852fb957f48ff3',
            'nonce' : str(time.time())
        }
    else:
        params = {                                             
        'apikey':'eb90f7ce6dad485c8a852fb957f48ff3',
        'market': market,
        'nonce': str(time.time())                   # use time in current epoch as nonce
    } 
    
    secret = '1b297b979337407bba31b00989dbdc19'     # secret to be included in signature
    url = urljoin(BASE_URL, PATH)                   # url without parameters (needed for api request)
    signature = signAPI(url, params, secret)        # returns the signature for api call 
    
    # request includes url, parameters as seen before, includes api signature in the request header
    r = requests.get(url, params = params, headers={'apisign':signature})
    if r.status_code == 200: 
        data = r.json()
        df = pd.DataFrame(data)
        print(df)
    else: 
        raise BittrexException(status_code = r.status_code(), data = r.json())
        
# run function to see output
get_orderhistory("btc-eth")

   success message                                             result
0     True          {'OrderUuid': 'f5dd374b-0b79-4f62-99ca-47faf6d...


### Retrieve withdrawal history
Requires an API Key with Read access 

Has parameters: 
* currency: string (not required) 

In [214]:
# GET /account/getwithdrawalhistory

def get_withdrawalhistory(currency = None): 
    PATH = '/api/v1.1/account/getwithdrawalhistory'
    
    # need to check if a specifc market is input
    if currency == None:
        params = {
            'apikey': 'eb90f7ce6dad485c8a852fb957f48ff3',
            'nonce' : str(time.time())
        }
    else:
        params = {                                             
        'apikey':'eb90f7ce6dad485c8a852fb957f48ff3',
        'currency': currency,
        'nonce': str(time.time())                   # use time in current epoch as nonce
    } 
    
    secret = '1b297b979337407bba31b00989dbdc19'     # secret to be included in signature
    url = urljoin(BASE_URL, PATH)                   # url without parameters (needed for api request)
    signature = signAPI(url, params, secret)        # returns the signature for api call 
    
    # request includes url, parameters as seen before, includes api signature in the request header
    r = requests.get(url, params = params, headers={'apisign':signature})
    if r.status_code == 200: 
        data = r.json()
        df = pd.DataFrame(data['result'])
        print(df)
    else: 
        raise BittrexException(status_code = r.status_code(), data = r.json())
        
# run function to see output
get_withdrawalhistory('BTC')

Empty DataFrame
Columns: []
Index: []


### Get Deposit History 
Requires API Key with Reaad access 

Has parameters: 
* currency: string (not required) 

In [212]:
# GET /account/getdeposithistory

def get_deposithistory(currency = None): 
    PATH = '/api/v1.1/account/getdeposithistory'
    
    # need to check if a specifc market is input
    if currency == None:
        params = {
            'apikey': 'eb90f7ce6dad485c8a852fb957f48ff3',
            'nonce' : str(time.time())
        }
    else:
        params = {                                             
        'apikey':'eb90f7ce6dad485c8a852fb957f48ff3',
        'currency': currency,
        'nonce': str(time.time())                   # use time in current epoch as nonce
    } 
    
    secret = '1b297b979337407bba31b00989dbdc19'     # secret to be included in signature
    url = urljoin(BASE_URL, PATH)                   # url without parameters (needed for api request)
    signature = signAPI(url, params, secret)        # returns the signature for api call 
    
    # request includes url, parameters as seen before, includes api signature in the request header
    r = requests.get(url, params = params, headers={'apisign':signature})
    if r.status_code == 200: 
        data = r.json()
        df = pd.DataFrame(data['result'])
        print(df)
    else: 
        raise BittrexException(status_code = r.status_code(), data = r.json())
        
# run function to see output
get_deposithistory()

         Id    Amount Currency  Confirmations            LastUpdated  \
0  97834192  0.197142      BTC              2  2019-08-28T01:50:44.8   

                                                TxId  \
0  b547f519d71bba3ec0bbf56ae14efa06c3683d12243032...   

                        CryptoAddress  
0  1DmzDwfZX2G2yZ7Y9P13xc6JcHSE9zZ2w8  
